In [96]:
#! /usr/bin/python3

def read_number(line, index):
    number = 0
    while index < len(line) and line[index].isdigit():
        number = number * 10 + int(line[index])
        index += 1
    if index < len(line) and line[index] == '.':
        index += 1
        decimal = 0.1
        while index < len(line) and line[index].isdigit():
            number += int(line[index]) * decimal
            decimal /= 10
            index += 1
    token = {'type': 'NUMBER', 'number': number}
    return token, index


def read_plus(line, index):
    token = {'type': 'PLUS'}
    return token, index + 1

def read_abs(line,index):
    token = {'type':'ABS'}
    return token,index+3

def read_int(line,index):
    token = {'type':'INT'}
    return token,index+3

def read_round(line,index):
    token = {'type':'ROUND'}
    return token,index+5

def read_minus(line, index):
    token = {'type': 'MINUS'}
    return token, index + 1

def read_times(line,index):
    token = {'type' : 'TIMES'}
    return token,index+1
def read_divide(line,index):
    token = {'type' : 'DIVIDE'}
    return token,index+1
def read_left(line,index):
    token = {'type' : 'LEFT'}
    return token,index+1
def read_right(line,index):
    token = {'type' : 'RIGHT'}
    return token,index+1

def calculate_brackets(tokens,index): #()内の計算
    sub = []
    pre = index+1
    while pre<len(tokens):
        if tokens[pre]['type'] == 'LEFT':
            sub.append(tokens[pre])
        elif tokens[pre]['type'] == 'RIGHT':
            if len(sub) == 0:
                break
            else:
                sub.pop()
        pre += 1
    sub_tokens = tokens[index+1:pre]
    sub_answer = evaluate(sub_tokens)
    return sub_answer,pre

def special_calculate(tokens,index): #abs(),round(),int()に使う
    pre = index+1
    (pre_answer_special,preindex) = calculate_brackets(tokens,pre)
    if tokens[index]['type']=='INT':
        pre_answer_special = int(pre_answer_special)
    elif tokens[index]['type']=='ABS':
        pre_answer_special = abs(pre_answer_special)
    elif tokens[index]['type'] == 'ROUND':
        pre_answer_special = round(pre_answer_special)
    return pre_answer_special,preindex




def pre_view(tokens,num,index):  #先読む
    pre_index = index+1
    pre_anwser = 0
    if tokens[pre_index]['type'] == 'TIMES':
        if tokens[pre_index+1]['type'] == 'NUMBER':
            pre_anwser = num * tokens[pre_index+1]['number']
            index += 2
        elif tokens[pre_index+1]['type'] == 'LEFT':
            (pre_anwser,index)=calculate_brackets(tokens,pre_index+1)
            pre_anwser = num*pre_anwser
            index+=1      
        else:
            (pre_answer,index) = special_calculate(tokens,pre_index+1) 
            pre_anwser = num*pre_anwser 
    elif tokens[pre_index]['type'] == 'DIVIDE':
        if tokens[pre_index+1]['type'] == 'NUMBER':
            pre_anwser = num / tokens[pre_index+1]['number']
            index +=2
        elif tokens[pre_index+1]['type'] == 'LEFT':
            (pre_anwser,index)=calculate_brackets(tokens,pre_index+1)
            pre_anwser = num/pre_anwser
            index+=1
        else:
            (pre_answer,index) = special_calculate(tokens,pre_index+1)
            pre_anwser = num/pre_anwser      
    else:
        pre_anwser = num
    if index+1 < len(tokens):
        if tokens[index+1]['type'] == 'TIMES' or tokens[index+1]['type'] == 'DIVIDE':
            return pre_view(tokens,pre_anwser,index)       
        else:
            return pre_anwser,index
    else:
        return pre_anwser,index

    
        
        



def tokenize(line):
    tokens = []
    index = 0
    while index < len(line):
        if line[index].isdigit():
            (token, index) = read_number(line, index)
        elif line[index] == '+':
            (token, index) = read_plus(line, index)
        elif line[index] == '-':
            (token, index) = read_minus(line, index)
        elif line[index] == '*':
            (token, index) = read_times(line, index)
        elif line[index] == '/':
            (token, index) = read_divide(line, index)
        elif line[index] == '(':
            (token, index) = read_left(line, index)
        elif line[index] == ')':
            (token,index) = read_right(line,index)
        elif line[index:index+3]=='abs':
            (token,index) = read_abs(line,index)
        elif line[index:index+3] =='int':
            (token,index) = read_int(line,index)
        elif line[index:index+5] == 'round':
            (token,index) = read_round(line,index)
        elif line[index] == ' ':
            index+=1

        else:
            print('Invalid character found: ' + line[index])
            exit(1)
        tokens.append(token)
    return tokens


    


def evaluate(tokens):
    answer = 0
    if tokens[0]['type']!='MINUS':
       tokens.insert(0, {'type': 'PLUS'}) # Insert a dummy '+' token
    index = 1    
    #負数から始める時'+'insert しない

    while index < len(tokens):
        pre_anwser = 0
        if tokens[index]['type'] == 'NUMBER':
            if tokens[index - 1]['type'] == 'PLUS':
                if index+1<len(tokens):
                    (pre_anwser,index) = pre_view(tokens,tokens[index]['number'],index)
                else:
                    pre_anwser = tokens[index]['number']
                answer += pre_anwser
            elif tokens[index - 1]['type'] == 'MINUS':
                if index+1<len(tokens):
                    (pre_anwser,index) = pre_view(tokens,tokens[index]['number'],index)
                else:
                    pre_anwser = tokens[index]['number']
                answer -= pre_anwser    
            elif tokens[index-1]['type'] == 'DIVIDE':
                answer /= tokens[index]['number']
            elif tokens[index-1]['type'] == 'TIMES':
                answer *= tokens[index]['number']
            else:
                print('Invalid syntax',index)
                exit(1)
            index += 1    
        elif tokens[index]['type'] == 'LEFT':
            (sub_answer,pre_index) = calculate_brackets(tokens,index)
            if tokens[index - 1]['type'] == 'PLUS':
                answer += sub_answer
            elif tokens[index - 1]['type'] == 'MINUS':
                answer -= sub_answer
            elif tokens[index - 1]['type'] == 'TIMES':
                answer *= sub_answer
            elif tokens[index - 1]['type'] == 'DIVIDE':
                answer /= sub_answer
            index +=pre_index+1
        elif tokens[index]['type'] == 'ABS' or tokens[index]['type'] == 'ROUND' or tokens[index]['type'] == 'INT':
            (sub_answer,pre_index)=special_calculate(tokens,index)
            if tokens[index - 1]['type'] == 'PLUS':
                answer += sub_answer
            elif tokens[index - 1]['type'] == 'MINUS':
                answer -= sub_answer
            elif tokens[index - 1]['type'] == 'TIMES':
                answer *= sub_answer
            elif tokens[index - 1]['type'] == 'DIVIDE':
                answer /= sub_answer
            index += pre_index
        index = index+1

    return answer


def test(line):
    tokens = tokenize(line)
    actual_answer = evaluate(tokens)
    expected_answer = eval(line)
    if abs(actual_answer - expected_answer) < 1e-8:
        print("PASS! (%s = %f)" % (line, expected_answer))
    else:
        print("FAIL! (%s should be %f but was %f)" % (line, expected_answer, actual_answer))


# Add more tests to this function :)
def run_test():
    print("==== Test started! ====")
    test("1+2")
    test("1.0+2.1-3")
    print("==== Test finished! ====\n")  
    test("1*2")
    test("1/2")
    test('1-2*3')
    test("1+2*2")
    test('1-2/2')
    test("1+2/2")
    test("1+2*2/2")
    test('1-2*2/2')
    test('2*2+3*3')
    test('3*3-2*3')
    test('(2)')
    test("1+(3+2)")
    test("3-(1+2)")
    test("1*(3*2)")
    test("1/(2/2)")
    test('1*(2+2*(1+1))')
    test('abs(-2)')
    test('int(3.3)')
    test('round(0.9)')
    test('12 + abs(int(round(-1.55) + abs(int(-2.3 + 4))))')

run_test()
while True:
    print('> ', end="")
    line = input()
    if line == 'esc':
        break
    tokens = tokenize(line)
    answer = evaluate(tokens)
    print("answer = %f\n" % answer)
    

==== Test started! ====
PASS! (1+2 = 3.000000)
PASS! (1.0+2.1-3 = 0.100000)
==== Test finished! ====

PASS! (1*2 = 2.000000)
PASS! (1/2 = 0.500000)
PASS! (1-2*3 = -5.000000)
PASS! (1+2*2 = 5.000000)
PASS! (1-2/2 = 0.000000)
PASS! (1+2/2 = 2.000000)
PASS! (1+2*2/2 = 3.000000)
PASS! (1-2*2/2 = -1.000000)
PASS! (2*2+3*3 = 13.000000)
PASS! (3*3-2*3 = 3.000000)
PASS! ((2) = 2.000000)
PASS! (1+(3+2) = 6.000000)
PASS! (3-(1+2) = 0.000000)
PASS! (1*(3*2) = 6.000000)
PASS! (1/(2/2) = 1.000000)
PASS! (1*(2+2*(1+1)) = 6.000000)
PASS! (abs(-2) = 2.000000)
PASS! (int(3.3) = 3.000000)
PASS! (round(0.9) = 1.000000)
PASS! (12 + abs(int(round(-1.55) + abs(int(-2.3 + 4)))) = 13.000000)
> 